# Evaluation Snorkel: Extration de partenariats d'entreprises
## Partie I: Corpus Preprocessing

In [80]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import pandas as pd
import pickle

from snorkel.models import StableLabel
from snorkel.db_helpers import reload_annotator_labels
from snorkel.parser import TSVDocPreprocessor
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence
from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import OrganizationMatcher, DictionaryMatch
from snorkel.viewer import SentenceNgramViewer

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Chargement des documents

### Noms des entreprises

On charge le noms des entreprises utiles pour le parsing qui ont été sauvegardés dans un fichier `pickle`

In [81]:
with open('./data/companies.pkl', 'rb') as f:
    company_names = pickle.load(f)
len(company_names)

872

### Configuration d'un `DocPreprocessor`

Nous allons charger le corpus de documents (dans notre cas les articles) et effectuer les pré-traitements sur les données. Nos articles qui sont stockés sous la forme : nom du document et texte associé. Nous allons utiliser la classe `TSVDocPreprocessor` pour lire ces documents.

In [82]:
n_docs = 1000
doc_preprocessor = TSVDocPreprocessor('data/articles.tsv', max_docs=n_docs)

### Creation d'un `CorpusParser`

Nous allons ensuite nous appuyer sur la librairie [Spacy](https://spacy.io/), un parser NLP, afin de découper nos documents en phrases et tokens et appliquer un NER (Named-entity Recognition) : classer les entités nommés dans des catégories (ex : Personne, Organisation, ...).

In [83]:
corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=n_docs)

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 9.87 s, sys: 4.54 s, total: 14.4 s
Wall time: 17.5 s


Les données sont stockées dans une base SQLLite par Snorkel. Nous pouvons faire quelques requêtes pour vérifier que nos données sont correctement enregistrées.

In [84]:
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 712
Sentences: 752


# 2. Génération des candidats

Dans cette partie nous allons extraire les _candidats__ de notre corpus. Ce sont les objets pour lesquels nous souhaitons faire une prédiction. 
Dans notre cas, un candidat est une paire de nom d'entreprise trouvée dans une phrase et pour lesquels nous voulons prédire s'ils sont partenaires ou non.

### Définition du schéma  `Candidate` 

Nous avons défini une relation binaire _competitor_ qui associe deux objets texte de type `Span`.

In [85]:
Partner = candidate_subclass('Partner', ['company1', 'company2'])

### Extraction des candidats

Nous allons maintenant extraire nos candidats en identifiant dans chaque phrase, les paires de n-grams (jusqu'à 5) qui sont tagués comme _Organisation_. 

In [86]:
ngrams         = Ngrams(n_max=5)
#partner_matcher = OrganizationMatcher()
company_matcher = DictionaryMatch(d=company_names)
#company_matcher = DictionaryMatch(d=company_names, stemmer='porter')
cand_extractor = CandidateExtractor(Partner, [ngrams, ngrams], [company_matcher, company_matcher])

Ensuite nous découpons nos données en jeu d'entrainement (80%), de developpements (10%) et de tests (10%).

In [87]:
docs = session.query(Document).order_by(Document.id).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 8:
            dev_sents.add(s)
        elif i % 10 == 9:
            test_sents.add(s)
        else:
            train_sents.add(s)

Nous appliquons ensuite notre extracteur de candidats sur les 3 jeux de données.

In [88]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(Partner).filter(Partner.split == i).count())

Clearing existing...
Running UDF...
[========================================] 100%

Number of candidates: 3978
Clearing existing...
Running UDF...
[========================================] 100%

Number of candidates: 359
Clearing existing...
Running UDF...
[========================================] 100%

Number of candidates: 446
CPU times: user 9.58 s, sys: 374 ms, total: 9.95 s
Wall time: 10.9 s


In [89]:
cands   = session.query(Partner).filter(Partner.split == 1).all()
SentenceNgramViewer(cands, session, n_per_page=1)

<IPython.core.display.Javascript object>

SentenceNgramViewer(cids=[[[304]], [[261]], [[28, 29, 30, 151, 152, 309]], [[263, 264, 265, 266, 285, 286, 287…

# 3. Gold labels : les données labelisées

Nous enregistrons maintenant les données qui ont été identifiés manuellement comme des partenaires. Ces données sont appelés les gold-labels et ils nous serviront à évaluer nos algorithmes.

In [101]:
def get_candidates_stable_ids(session, candidate_class, split, annotator_name='gold') :
    # Get split candidates
    candidates = session.query(candidate_class).filter(
        candidate_class.split == split
    ).all()
    stables_id_list = []
    for c in candidates:
        context_stable_ids = '~~'.join(x.get_stable_id() for x in c)
        stables_id_list.append(context_stable_ids)
    return stables_id_list
    

In [102]:
FPATH = 'data/gold_labels.tsv'

def load_external_labels(session, candidate_class, split, annotator_name='gold'):
    gold_labels = pd.read_csv(FPATH, sep="\t")
    candidates_stables_id_list = get_candidates_stable_ids(session, candidate_class, split, annotator_name)
    nb = 0
    for index, row in gold_labels.iterrows(): 
        non_trouve = False
        # We check if the label already exists, in case this cell was already executed
        context_stable_ids = "~~".join([row['company1'], row['company2']])
    
        if context_stable_ids not in candidates_stables_id_list :
            non_trouve = True
        
        query = session.query(StableLabel).filter(StableLabel.context_stable_ids == context_stable_ids)
        query = query.filter(StableLabel.annotator_name == annotator_name)
        if query.count() == 0:
            session.add(StableLabel(
                context_stable_ids=context_stable_ids,
                annotator_name=annotator_name,
                value=row['label']))
                    
        # Because it's a symmetric relation, load both directions...
        context_stable_ids = "~~".join([row['company2'], row['company1']])
        
        if non_trouve and context_stable_ids not in candidates_stables_id_list :
            #print(context_stable_ids)
            nb += 1
        query = session.query(StableLabel).filter(StableLabel.context_stable_ids == context_stable_ids)
        query = query.filter(StableLabel.annotator_name == annotator_name)
        if query.count() == 0:
            session.add(StableLabel(
                context_stable_ids=context_stable_ids,
                annotator_name=annotator_name,
                value=row['label']))
    # Commit session
    session.commit()
    
    print("Total non trouvé", nb)

    # Reload annotator labels
    #reload_annotator_labels(session, candidate_class, annotator_name, split=0, filter_label_split=False)
    #reload_annotator_labels(session, candidate_class, annotator_name, split=1, filter_label_split=False)
    reload_annotator_labels(session, candidate_class, annotator_name, split=split, filter_label_split=False)

In [103]:
%time missed = load_external_labels(session, Partner, 1, annotator_name='gold')

Total non trouvé 1621
AnnotatorLabels created: 0
CPU times: user 46.3 s, sys: 1.7 s, total: 48 s
Wall time: 49 s


# 4. Vérifications

On fait quelques requêtes pour s'assurer que nos données sont correctement enregistrées.

In [14]:
cands = session.query(Partner).filter(Partner.split == 1).all()

In [15]:
print (cands[0].company1)
print (cands[0].company2)

Span("b'Toyota'", sentence=853, chars=[0,5], words=[0,0])
Span("b'Azure'", sentence=853, chars=[68,72], words=[9,9])


In [16]:
sentence = cands[0].get_parent()
print(sentence)
document = sentence.get_parent()
print(document)

Sentence(Document 4469d5f8-9801-48f7-ae9b-7f45ed87141d,0,b'Toyota Connected will leverage Microsoft\xe2\x80\x99s cloud computing platform Azure to analyse data and assist in developing new products for drivers, businesses with car fleets, and dealers\n')
Document 4469d5f8-9801-48f7-ae9b-7f45ed87141d


In [17]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

In [18]:
get_text_between(cands[0])

' Connected will leverage Microsoft’s cloud computing platform '

In [19]:
cands[0][0].get_attrib_tokens(a='words')

['Toyota']

In [20]:
cands[0][1].get_attrib_tokens(a='words')

['Azure']